In [6]:
from PIL import Image, ImageFilter
import PIL
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import cv2
import pickle
import numpy as np

from gen_pic_utils import *
import populate_library

from math import floor
import random

In [7]:
def save(filename,data):
    with open(filename, 'wb+') as f:
        pickle.dump(data, f,protocol=0)
        
def load(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
        f.close()
        return data

In [8]:
def box_to_trapezoid(sample,trapezoid):
    '''Tranform 2d box-sample into trapezoid (car displacement area)'''
    
    trapezoid = np.array(trapezoid)        
    box = np.array([[0,0],[0,1],[1,1],[1,0]])
    h, status = cv2.findHomography(np.float_(box), np.float_(np.array(trapezoid)))
    
    sample = np.float32([sample]).reshape(-1,1,2)
    sample = cv2.perspectiveTransform(sample,h)
    
    return sample    

In [80]:
def draw_sample(road_type, sample):
    
    lib = populate_library.populateLibrary()     
    road = lib.getElement("roads", road_type) 
    
    configs = load('scene_configs_py2')
    polygon = configs[road_type][0]
    sample = box_to_trapezoid(sample,polygon)    

    draw = ImageDraw.Draw(road.data)
    draw.polygon(polygon, fill=None, outline=(255,0,0,255))      
    
    eps = 5
    x = sample[0][0][0]
    y = sample[0][0][1] 
    draw.ellipse((x-eps, y-eps, x+eps, y+eps), fill = 'blue', outline ='blue')

    display.display(road.data)
    

In [68]:
def next_prime():
    '''Get next prime number'''
    def is_prime(num):
        "Checks if num is a prime value"
        for i in range(2,int(num**0.5)+1):
            if(num % i)==0: return False
        return True
 
    prime = 3
    while(1):
        if is_prime(prime):
            yield prime
        prime += 2


def vdc(n, base=2):
    '''Van der Corput sequence''' 
    vdc, denom = 0, 1
    while n:
        denom *= base
        n, remainder = divmod(n, base)
        vdc += remainder/float(denom)
    return vdc

def halton_sequence(size, dim):
    '''Halton sequence'''
    seq = []
    primeGen = next_prime()
    next(primeGen)
    for d in range(dim):
        base = next(primeGen)
        seq.append([vdc(i, base) for i in range(size)])
    return seq

def halton_sample(i, dim):
    '''i-th Halton sample'''
    seqs = halton_sequence(i, dim)
    
    sample = []
    for seq in seqs:
        sample += [seq[-1]]
    
    return sample

def random_sample(dim):
    '''uniform random sample'''
    sample = []
    for i in range(dim):
        sample += [random.random()]
        
    return sample


def get_sample(i, dim, method, types=[], domains=[]):
    '''Get a sample and adapt it to the domain'''
    
    # Sample
    if method == 'random':
        sample = random_sample(dim)
    if method == 'halton':
        sample = halton_sample(i, dim)
        
    # Adapt sample to type and domain
    assert len(types) == len(domains)

    
    for j in range(len(sample)):
        domain = domains[j]
        typ = types[j]
        
        sample[j] = sample[j]*(domain[1]-domain[0])+domain[0]
        
        if typ == 'int':
            sample[j] = int(sample[j])
        elif typ == 'float':
            sample[j] = float(sample[j])
            
    return sample
    
